In [1]:
# Import required libraries
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException, TimeoutException, NoSuchElementException, StaleElementReferenceException, ElementClickInterceptedException
from datetime import datetime
import time
import os
import wget
import json
import csv

In [2]:
# Set up ChromeDriver
service = Service('E:/Programs/Chrome Driver/chromedriver.exe')
driver = webdriver.Chrome(service=service)

In [3]:
import pickle

# Define the file path to load the data
file_path = 'hrefs_with_categories.pkl'

# Load the data using pickle
with open(file_path, 'rb') as file:
    hrefs_with_categories = pickle.load(file)

In [4]:
for href, category in hrefs_with_categories:
    if category.lower() == "politics":
        driver.get(href)
        break 

In [5]:
# Find the "more top stories" element and click it
more_button_xpath = '//*[contains(concat( " ", @class, " " ), concat( " ", "top-newslist", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "more-btn", " " ))]//span'

more_button = driver.find_element(By.XPATH, more_button_xpath)
more_button.click()

In [6]:
# List to store href and corresponding last updated values
hrefs_all = []

# Base URL for the first page
base_url = "https://timesofindia.indiatimes.com/politics/news"
suffix = 1  # Start without a suffix for the first page

def extract_last_updated(li):
    """Extract the last updated value from a list item element."""
    try:
        return li.find_element(By.XPATH, './/*[contains(concat( " ", @class, " " ), concat( " ", "strlastupd", " " ))]').text.strip()
    except NoSuchElementException:
        return None

while suffix <= 51:  # Limit the navigation to page 51
    if suffix == 1:
        url = base_url
    else:
        url = f"{base_url}/{suffix}"
    
    print(f"Visiting: {url}")
    
    try:
        driver.get(url)
        li_elements = driver.find_elements(By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "top-newslist", " " ))]//li')
        
        for li in li_elements:
            try:
                a_tag = li.find_element(By.XPATH, './/a')
                href = a_tag.get_attribute('href')
                last_updated_raw = extract_last_updated(li)
                
                # Collect href and last updated values regardless of the days ago calculation
                if last_updated_raw:
                    hrefs_all.append((href, last_updated_raw))
                
            except NoSuchElementException:
                print(f"Skipping li element due to missing href or last updated value.")
                continue

        suffix += 1
        time.sleep(1)
        
    except NoSuchElementException:
        print(f"No more pages found after page {suffix}. Stopping.")
        break

# Output all collected hrefs with last updated values
print("Collected hrefs with last updated values:")
for href, last_updated in hrefs_all:
    print(f"Href: {href}, Last Updated: {last_updated}")


Visiting: https://timesofindia.indiatimes.com/politics/news
Visiting: https://timesofindia.indiatimes.com/politics/news/2
Visiting: https://timesofindia.indiatimes.com/politics/news/3
Visiting: https://timesofindia.indiatimes.com/politics/news/4
Visiting: https://timesofindia.indiatimes.com/politics/news/5
Visiting: https://timesofindia.indiatimes.com/politics/news/6
Visiting: https://timesofindia.indiatimes.com/politics/news/7
Visiting: https://timesofindia.indiatimes.com/politics/news/8
Visiting: https://timesofindia.indiatimes.com/politics/news/9
Visiting: https://timesofindia.indiatimes.com/politics/news/10
Visiting: https://timesofindia.indiatimes.com/politics/news/11
Visiting: https://timesofindia.indiatimes.com/politics/news/12
Visiting: https://timesofindia.indiatimes.com/politics/news/13
Visiting: https://timesofindia.indiatimes.com/politics/news/14
Visiting: https://timesofindia.indiatimes.com/politics/news/15
Visiting: https://timesofindia.indiatimes.com/politics/news/16
Vis

In [11]:
# Create a list of pairs
list_of_pairs = [{"href": href, "last_updated": last_updated} for href, last_updated in hrefs_all]

# Output the list of pairs
print("List of pairs:")
for pair in list_of_pairs:
    print(pair)

List of pairs:
{'href': 'https://timesofindia.indiatimes.com/politics/news/tipra-motha-chief-meets-congress-leaders-as-bjp-eyes-pact/articleshow/107980204.cms', 'last_updated': '216 DAYS AGO'}
{'href': 'https://timesofindia.indiatimes.com/politics/news/ahmed-patels-legacy-wont-go-in-vain-mumtaz-reacts-after-congress-announces-giving-late-fathers-seat-to-aap/articleshow/107966185.cms', 'last_updated': '217 DAYS AGO'}
{'href': 'https://timesofindia.indiatimes.com/politics/news/buzz-over-navjot-singh-sidhu-return-to-bjp-gurdaspur-ticket-for-yuvraj-singh/articleshow/107882378.cms', 'last_updated': '219 DAYS AGO'}
{'href': 'https://timesofindia.indiatimes.com/politics/news/victory-of-voters-sharad-pawar-after-supreme-court-ruling-on-his-factions-name-symbol/articleshow/107829961.cms', 'last_updated': '222 DAYS AGO'}
{'href': 'https://timesofindia.indiatimes.com/politics/news/supreme-court-allows-sharad-faction-to-use-ncp-sharadchandra-pawar-name/articleshow/107825594.cms', 'last_updated': '

In [12]:
import pickle

# Define the file path to save the data (e.g., articles_from_politics.pkl)
file_path = 'articles_from_politics.pkl'

# Save the data using pickle
with open(file_path, 'wb') as file:
    pickle.dump(list_of_pairs, file)

In [10]:
driver.quit()